# Creating Protein Meshes in Graphein & 3D Visualisation

Graphein provides functionality to create meshes of protein surfaces. The mesh computation is handled under the hood by PyMol to produce `.obj` files, which are handled by many geometric analysis libraries (such as [PyTorch3D](https://pytorch3d.org/)). We'll create a few protein meshes, convert them to PyTorch3D objects and visualise them. Again, there is a high-level and low-level API for convenience. We'll be looking at PDB [1LDF](https://www.rcsb.org/structure/1ldf), an elegant structure of the E. col glycerol uptake facilitator protein.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/a-r-j/graphein/blob/master/notebooks/protein_mesh_tutorial.ipynb)

**NB** lots of features of PyTorch3D are GPU only. If on macOS (only!), you can install via pip. Ensure you have PyTorch `1.9.0`. This is the only supported configuration. See: https://github.com/facebookresearch/pytorch3d/blob/main/INSTALL.md

In [ ]:
# Install Graphein if necessary
# !pip install graphein

# Install pymol if necessary - in this tutorial PyMol is only used for the initial plot. Feel free to skip!
# sudo apt-get install pymol (recommended for colab) OR conda install -c schrodinger pymol

In [ ]:
# Install torch 1.9.0
# pip install torch==1.9.0

# Install pytorch3d
# pip install pytorch3d

In [ ]:
# NBVAL_SKIP
%matplotlib inline
import logging
logging.basicConfig(level=logging.CRITICAL)

## High-level API
### Config
Again, we use a [ProteinMeshConfig](https://graphein.ai/modules/graphein.protein.html#graphein.protein.config.ProteinMeshConfig) config object to manage global configurations.

* `pymol_command_line_options`: String of additional pymol command line options when launching. A full list can be found [here](https://www.pymolwiki.org/index.php/Command_Line_Options). The default options, `"-cKq"`, do:
   * `c`: launch in command-line only mode for batch processing
   * `K`: keep alive: when running without a GUI, don't quit after the input
            is exhausted
    * `q`: supress startup message 
    
* `pymol_commands` : List of string pymol commands to execute. You can use these to configure the exact mesh you wish to construct. A reference to these can be found [here](https://pymol.org/pymol-command-ref.html)

In [ ]:
from graphein.protein.config import ProteinMeshConfig
config = ProteinMeshConfig()
config.dict()

### Building a mesh

Let's build a mesh!

In [ ]:
# NBVAL_SKIP
from graphein.protein.meshes import create_mesh

verts, faces, aux = create_mesh(pdb_code="1ldf", config=config)
print(verts, faces, aux)

In order to visualise these tensors containing vertices, faces and aux data, we convert them into a PyTorch3D `Mesh` object

In [ ]:
# NBVAL_SKIP
from graphein.protein.meshes import convert_verts_and_face_to_mesh
m = convert_verts_and_face_to_mesh(verts, faces)

from graphein.protein.visualisation import plot_pointcloud
plot_pointcloud(m, title="my first mesh")

## Modifying Mesh Parameters

Let's see what happens if we play around with some of the parameters and make a smoother mesh using

In [ ]:
# NBVAL_SKIP
pymol_commands = {"pymol_commands": ["hide cartoon",
                                     "set solvent_radius, 10",
                                     "alter all, vdw=4",
                                     "sort",
                                     "set surface_quality, 1",
                                     "show surface"]}

config = ProteinMeshConfig(**pymol_commands)

verts, faces, aux = create_mesh(pdb_code="1ldf", config=config)

m = convert_verts_and_face_to_mesh(verts, faces)

plot_pointcloud(m, title="my second mesh")

## Using PyMol for 3D Visualisation
We also provide a wrapper for pymol (based on [IPyMol](https://github.com/cxhernandez/ipymol) developed by [@cxhernandez](https://github.com/cxhernandez))

In [ ]:
# NBVAL_SKIP
from graphein.utils.pymol import viewer as pymol
pymol.delete("all") # delete all objects from other sessions if necessary.
pymol.fetch("1ldf")
pymol.display()
